In [7]:
#ONLY RUN THIS PART IF YOU WANT THE WHOLE THING. If you want to understand what the code does and see the runtimes for
# each function, running each function separately will be better 
from bs4 import BeautifulSoup
import urllib
from datetime import datetime
from datetime import timedelta
import pandas as pd
import numpy as np



#function to get the links from https://www.reuters.com/resources/archive/us/DATE.html
#DATE is in YYYYMMDD format
#get two dates from user:start date and end date. Scrape all articles between these dates 
def get_links_nyt():
    #get input
    start_date = datetime.strptime(input("Please enter the start date:"), '%Y%m%d')
    end_date=datetime.strptime(input("Please enter the end date:"), '%Y%m%d')
    #create list of dates between start and end dates
    delta=end_date-start_date
    date_list=[start_date + timedelta(i) for i in range(delta.days + 1)]
    link_list=[]
    #scrape what the links of articles for each date. put them in a list.
    for i in date_list:
        try:
            p=urllib.request.urlopen(' https://www.nytimes.com/issue/todayspaper/'+i.strftime('%Y/%m/%d/')+'todays-new-york-times').read()  
        except Exception as e:
            link_list.append(e)
            continue
        
        soup = BeautifulSoup(p,'lxml')
        content=soup.find_all("h2",class_="headline") 
        print(len(content))
        for i in content:
            link_list.append(i.a['href'])            
            
                
    return link_list


def get_dates_nyt(links):
    dates_list=[]
    for i in links:
        try:
            y=urllib.request.urlopen(i).read()
        except Exception as e:
            dates_list.append(e)
            continue
        z = BeautifulSoup(y,'lxml')
        a_dates=z.find_all('meta',attrs={'itemprop':'datePublished'})
        print(a_dates)
        if len(a_dates)>=1:
            dates_list.append(a_dates[0]['content'].lower().split('T'))
        elif len(a_dates)==0:
            dates_list.append("Na")                        
    return dates_list

def get_keywords_nyt(links):
    keywords_list=[]
    for i in links:
        try:
            y=urllib.request.urlopen(i).read()
        except Exception as e:
            keywords_list.append(e)
            continue
        z = BeautifulSoup(y,'lxml')
        a_keywords=z.find_all('meta',attrs={'name':'news_keywords'})
        if len(a_keywords)>=1:
            keywords_list.append(a_keywords[0]['content'].lower())
        elif len(a_keywords)==0:
            keywords_list.append("Na")                        
    return keywords_list


def get_articles_nyt(links):
    article_list=[]
    for i in links:
        try:
            y=urllib.request.urlopen(i).read()
       
        except Exception as e:
            article_list.append(e)
            continue
        
        z = BeautifulSoup(y,'lxml')
        a=z.find_all("div",class_="css-18sbwfn StoryBodyCompanionColumn")
        if len(a)>=1:
            article_list.append(' '.join(paragraph.get_text() for paragraph in a))
        elif len(a)==0:
            article_list.append('Na')
    return article_list

def get_titles_nyt(links):
    title_list=[]
    for i in links:
        try:
            y=urllib.request.urlopen(i).read()     
        except Exception as e:
            title_list.append(e)
            continue
        z = BeautifulSoup(y,'lxml')
        a_titles=z.find_all("title")
        if len(a_titles)>=1:
            title_list.append(a_titles[0].get_text()[0:-21])
        elif len(a)==0:
            title_list.append("Na")            
    return title_list



#links

In [16]:
links=get_links()

['https://www.nytimes.com/2018/05/15/world/asia/north-korea-postpones-talks.html', 'https://www.nytimes.com/2018/05/15/us/cia-hacking-tools-leak.html', 'https://www.nytimes.com/2018/05/15/world/middleeast/israelis-gaza-protests.html', 'https://www.nytimes.com/2018/05/15/obituaries/tom-wolfe-pyrotechnic-nonfiction-writer-and-novelist-dies-at-88.html', 'https://www.nytimes.com/2018/05/15/world/europe/meghan-markle-prince-harry-family.html', 'https://www.nytimes.com/2018/05/15/us/politics/gina-haspel-cia-torture-letter-senate.html', 'https://www.nytimes.com/2018/05/15/us/cambridge-analytica-federal-investigation.html', 'https://www.nytimes.com/2018/05/15/us/national-guard-texas-mexico.html', 'https://www.nytimes.com/2018/05/15/us/parkland-parents-school-board.html', 'https://www.nytimes.com/2018/05/15/world/asia/malaysia-najib-razak-fall.html', 'https://www.nytimes.com/2018/05/15/books/anne-frank-diary-new-pages.html', 'https://www.nytimes.com/2018/05/15/nyregion/marijuana-arrests-mayor-d

In [24]:
with open("links_nyt_20180516_20180621.txt", "w") as f:
    for s in links:
        f.write(str(s) +"\n")

In [35]:
with open('dates_nyt_201805160621.pkl', 'wb') as f:
    pickle.dump(dates, f)


In [61]:
articles=get_articles_nyt(links)

In [71]:
keywords=get_keywords_nyt(links)

In [96]:
titles=get_titles_nyt(links)

In [98]:
df_nyt = pd.DataFrame(np.column_stack([dates, links, keywords,articles,titles]),columns=['date', 'link', 'keywords','article','title']) 

In [108]:
df_nyt.to_csv("20180516-20180621_nyt.csv")

In [12]:
string.punctuation=string.punctuation.replace('”', '')
string.punctuation=string.punctuation.replace('—', '')
string.punctuation
#test=pd.read_csv('20180516-20180621_nyt.csv')

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
import string

len(string.punctuation)
#string.punctuation+='”'
#string.punctuation+='—'
trantab=str.maketrans(string.punctuation,'                                  ')

wordcount_test=test['article'][0].translate(trantab)

' '.join(wordcount_test.split())

'SEOUL South Korea North Korea threw President Trump’s planned summit meeting with its leader Kim Jong un into doubt on Wednesday threatening to call off the landmark encounter if the United States insisted on “unilateral nuclear abandonment The warning made by the North’s disarmament negotiator caught Trump administration officials off guard and set off an internal debate over whether Mr Kim was merely posturing in advance of the meeting in Singapore next month or was erecting a serious new hurdle The abrupt change in tone began early Wednesday when North Korea indefinitely postponed high level talks with South Korea over the North’s sudden objection to joint military drills by the South and the United States that began last week The North also raised the possibility of scrapping the meeting with Mr Trump Then hours later the North broadened the source of its anger and sharpened the threat to the summit with Mr Trump Kim Kye kwan a vice foreign minister rejected the administration’s d

In [173]:
import os
os.chdir('/Users/berkayadlim/Desktop/NLP_Research/PICKLE!')

In [175]:
import pickle
df_reuters = pickle.load( open( "master_df.pkl", "rb" ) )

In [193]:
df_reuters=df_reuters.reset_index(drop=True)

In [198]:
df_reuters_unique=df_reuters.drop_duplicates('article')

In [200]:
df_reuters_unique.to_csv('20180516-20180621_reuters_unique.csv')

In [206]:
df_nyt_unique=test.drop_duplicates('article')

In [208]:
df_nyt_unique.to_csv('20180516-20180621_nyt_unique.csv')

In [211]:
uniques=pd.read_csv('20180516-20180621_nyt_unique.csv')

In [214]:
with open('20180516-20180621_reuters_unique.pkl', 'wb') as f:
    pickle.dump(df_reuters_unique, f)


In [3]:
import os
os.chdir('/Users/berkayadlim/Desktop/NLP_Research/PICKLE!')

In [16]:
wordcount_test=wordcount_test.replace('“', '')

In [17]:
wordcount_test

'SEOUL  South Korea   North Korea threw President Trump’s planned summit meeting with its leader  Kim Jong un  into doubt on Wednesday  threatening to call off the landmark encounter if the United States insisted on unilateral nuclear abandonment  The warning  made by the North’s disarmament negotiator  caught Trump administration officials off guard and set off an internal debate over whether Mr  Kim was merely posturing in advance of the meeting in Singapore next month or was erecting a serious new hurdle The abrupt change in tone began early Wednesday  when North Korea indefinitely postponed high level talks with South Korea over the North’s sudden objection to joint military drills by the South and the United States that began last week  The North also raised the possibility of scrapping the meeting with Mr  Trump  Then hours later  the North broadened the source of its anger and sharpened the threat to the summit with Mr  Trump  Kim Kye kwan  a vice foreign minister  rejected the 

In [1]:
len(article_list_nyt)

NameError: name 'article_list_nyt' is not defined